In [35]:
# RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=None,  bootstrap=True, n_jobs=-1, random_state=0)
# SVC(C=1.0, kernel='linear', tol=0.001, cache_size=2000, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
# NearestNeighbors(n_neighbors=10, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)
# MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

In [1]:
import os
import re
import json
import exrex
import numpy
import Bio.SeqIO
import pandas as pd
from sklearn.svm import SVC
from joblib import Parallel, delayed
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [2]:
# Function to convert degenerative k-mers into a list of k-mers 
def convert_degenerative_k_mers(k_mer):
	k_mers = []
	temp_k_mer = ""
	
	for n in range(len(k_mer)): 
		if k_mer[n] == "A": temp_k_mer = temp_k_mer + "[A]"
		if k_mer[n] == "C": temp_k_mer = temp_k_mer + "[C]"
		if k_mer[n] == "G": temp_k_mer = temp_k_mer + "[G]"
		if k_mer[n] == "T": temp_k_mer = temp_k_mer + "[T]"
		if k_mer[n] == "N": temp_k_mer = temp_k_mer + "[ACGT]"
		if k_mer[n] == "X": temp_k_mer = temp_k_mer + "[ACGT]"
		if k_mer[n] == "V": temp_k_mer = temp_k_mer + "[^ACG]"
		if k_mer[n] == "H": temp_k_mer = temp_k_mer + "[^ACT]"
		if k_mer[n] == "D": temp_k_mer = temp_k_mer + "[^AGT]"
		if k_mer[n] == "B": temp_k_mer = temp_k_mer + "[^CGT]"
		if k_mer[n] == "M": temp_k_mer = temp_k_mer + "[AC]"
		if k_mer[n] == "W": temp_k_mer = temp_k_mer + "[AT]"
		if k_mer[n] == "R": temp_k_mer = temp_k_mer + "[AG]"
		if k_mer[n] == "K": temp_k_mer = temp_k_mer + "[GT]"
		if k_mer[n] == "S": temp_k_mer = temp_k_mer + "[CG]"
		if k_mer[n] == "Y": temp_k_mer = temp_k_mer + "[CT]"

	for i in exrex.generate(temp_k_mer): 
		k_mers.append(i)
	return k_mers

In [3]:
# Function to load test data from a fasta file
def loadDataTrain(file_path):
	# Initialize the data matrix
	D = []
	# Iterate through the fasta file
	for record in Bio.SeqIO.parse(file_path, "fasta"):
		# If there is a class label, save id, sequence and class label in the data list
		try: 
			indexes = [i for i, c in enumerate(record.description) if c == "|"]
			D.append([record.description, str(record.seq.upper()).replace('N',''), record.description[indexes[len(indexes)-1] +1 :]])
		# If there is a no class label, save id and sequence in the data list
		except: D.append([record.descrition, str(record.seq.upper())])
	# Return the data matrix
	return D

In [4]:
# Function to load test data from a fasta file
def loadDataTest(file_path, D_train):
	# Load train data to remove it from test data
	id = [d[0] for d in D_train]
	D_train.clear()
	# Initialize the data matrix
	D = []
	# Iterate through the fasta file
	for record in Bio.SeqIO.parse(file_path, "fasta"):
		# If there is a class label, save id, sequence and class label in the data list
		try: 
			# Get index of last separator
			indexes = [i for i, c in enumerate(record.description) if c == "|"]
			# Save id, sequence and class label
			if record.description not in id:
				D.append([record.description, str(record.seq.upper()), record.description[indexes[len(indexes)-1] +1 :]])
		# If there is a no class label, save id and sequence in the data list
		except: D.append([record.descrition, str(record.seq.upper())])
	# Return the data matrix
	return D

In [5]:
# Function to compute the occurence vector of sequence
def computeSequenceVector(d, K, k):
	# Generate an empty dictionary
	x = {}
	# Initialize the dictionary with targets as keys and 0 as value
	x = x.fromkeys(K.keys(), 0)
	# Go through the sequence 
	for i in range(0, len(d[1]) - k + 1, 1):
		# Try to increment the current k-mer value by 1
		try: x[d[1][i:i + k]] = x[d[1][i:i + k]] + 1
		# Pass if the k-mers does not exist
		except: pass
	# Return the vector and associated target
	return [list(x.values()), d[2]]

In [6]:
# Function to generate the samples matrix (X) and the target values (y)
def generateSamplesTargets(D, K, k):
	# Samples matrix
	X = []
	# Target values
	y = []
	# Iterate through the data
	data = Parallel(n_jobs = -1)(delayed(computeSequenceVector)(d, K, k) for d in D)
	# Add to the matrices
	for d in data: 
		X.append(d[0])
		y.append(d[1])

	# Convert to numpy array format
	X = numpy.asarray(X)
	y = numpy.asarray(y)
	# Return the samples matrix (X) and the target values (y)
	return X, y

In [7]:
# Compute and save performance metrics for STREME
data = {}

# Define the list of variants
variants = ["Alpha", "Beta", "Delta", "Epsilon", "Eta", "Gamma", "Iota", "Kappa", "Lambda", "Omicron"]
# Iterate throught prediction files
for n in range(1, 101): 
    y_true = []
    y_pred = []
    k_mers = []
    for variant in variants:
        with open("MEME_ZOOPS/" + str(n) + "/" + variant +  "/meme.txt") as file:
            for line in file: 
                if line.strip().count("MOTIF") and line.strip().count("width =   9"): 
                    line = line.strip()
                    start = 6
                    end = start + 9
                    k_mer =  line[start:end]
                    if bool(re.match('^[ACGT]+$', k_mer)) == True: 
                       k_mers.append(k_mer)
                    else: 
                        temp_k_mers = convert_degenerative_k_mers(k_mer)
                        for temp_k_mer in temp_k_mers: 
                            k_mers.append(temp_k_mer)
    k_mers = set(k_mers)

    K = dict.fromkeys(k_mers, 0)
    D_train = loadDataTrain("data/SARS-CoV-2_train_" + str(n) + ".fasta")
    X_train, y_train = generateSamplesTargets(D_train, K , 9)

    D_test = loadDataTest("data/SARS-CoV-2.fasta", D_train)
    X_test, y_test = generateSamplesTargets(D_test, K , 9)

    model = SVC(kernel = 'linear', C = 1, cache_size = 1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Compute the performance metrics
    f1 = f1_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    print(str(n) +") precision", precision, "recall", recall, "f1_score", f1)

    # Save the predictions
    file = open("MEME_ZOOPS/" + str(n) + "/prediction.csv", "w")
    file.write("id,y_pred\n")
    for i, y in enumerate(y_pred): file.write(D_test[i][0] + "," + y + "\n")
    file.close()


1) precision 0.8138297493164192 recall 0.8117761905213605 f1_score 0.7237909561051737
2) precision 0.8205162903855114 recall 0.835295503237537 f1_score 0.8268446081517457
3) precision 0.9665179405367246 recall 0.9949846180101426 f1_score 0.9800767963958883
4) precision 0.9471666018970915 recall 0.9928995322566697 f1_score 0.9633432367467659
5) precision 0.8202493989539434 recall 0.9140218833285093 f1_score 0.8416836656385358


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


6) precision 0.9664725361116966 recall 0.992531959522928 f1_score 0.9787590297777988


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


7) precision 0.8974702682056422 recall 0.9804613234634825 f1_score 0.9072820835317662


...

97) precision 0.8833583953613549 recall 0.9932175964420006 f1_score 0.912642855488303


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


98) precision 0.7771852837256192 recall 0.9075195701938604 f1_score 0.8010711254197013
99) precision 0.7250626219653056 recall 0.8203432869672989 f1_score 0.7427327844675851


c:\Users\lebat\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


100) precision 0.9015387258591085 recall 0.9768536047937811 f1_score 0.8954995283769
